# Edge Classifier Debug

**Goal**: Test the weighting and hard cut config of the data loading process

In [1]:
%load_ext autoreload
%autoreload 2

import os
import yaml

import numpy as np
import pandas as pd
import seaborn as sns
import yaml
import torch
from torch_geometric.data import Data
import matplotlib.pyplot as plt

from time import time as tt

import sys
print(sys.path)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.8/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: libtorch_cuda_cpp.so: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/opt/conda/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.8/site-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


['/workspaces/CommonFramework/notebooks/testing', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/torchtext-0.12.0a0-py3.8-linux-x86_64.egg', '/workspaces/CommonFramework']


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.version.cuda)
print(torch.cuda.device_count())
print(torch.cuda.get_device_properties("cuda:0"))
print(torch.cuda.get_device_name("cuda:0"))
print(torch.__version__)
print(torch.cuda.get_arch_list())

True
0
11.6
1
_CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 Super with Max-Q Design', major=7, minor=5, total_memory=8191MB, multi_processor_count=48)
NVIDIA GeForce RTX 2080 Super with Max-Q Design
1.12.0a0+2c916ef
['sm_52', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'compute_86']


In [3]:
import torch_scatter
import torch_spline_conv

In [4]:
crow_indices = [0, 2, 4]
col_indices = [0, 1, 0, 1]
values = [1, 2, 3, 4]
torch.sparse_csr_tensor(torch.tensor(crow_indices,dtype=torch.int64),torch.tensor(col_indices, dtype=torch.int64),torch.tensor(values),dtype=torch.double)

tensor(crow_indices=tensor([0, 2, 4]),
       col_indices=tensor([0, 1, 0, 1]),
       values=tensor([1., 2., 3., 4.]), size=(2, 2), nnz=4,
       dtype=torch.float64, layout=torch.sparse_csr)

In [5]:
import torch
from torch_scatter import scatter_max

src = torch.tensor([[2, 0, 1, 4, 3], [0, 2, 1, 3, 4]])
index = torch.tensor([[4, 5, 4, 2, 3], [0, 0, 2, 2, 1]])

out, argmax = scatter_max(src, index, dim=-1)

In [8]:
import torch_sparse

OSError: /opt/conda/lib/python3.8/site-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs

In [7]:
import numba
import cudf

## GNN Debug

In [7]:
from gnn4itk_cf.stages.edge_classifier.models.interaction_gnn import InteractionGNN

NOTE! Installing ujson may make loading annotations faster.


In [9]:
config = yaml.load(open('trackml_gnn_train.yaml'), Loader=yaml.FullLoader)

In [10]:
model = InteractionGNN(config)

In [11]:
model.setup('fit')

AssertionError: No data files found in /global/cfs/cdirs/m3443/data/GNN4ITK/CommonFrameworkExamples/Example_3_Dev/metric_learning/trainset

In [ ]:
sample = model.trainset[0]

In [ ]:
sample

In [ ]:
graph_sizes = [sample.edge_index.shape[1] for sample in model.trainset]

In [ ]:
sns.distplot(graph_sizes)

## Metric Debug

In [ ]:
%%time
with torch.no_grad():
    out = model.to("cuda")(sample.to("cuda"))

In [ ]:
sns.distplot(torch.sigmoid(out).cpu())

In [ ]:
sample

In [ ]:
sample.weights.shape

In [ ]:
(sample.weights==0).sum(), (sample.weights==1).sum(), (sample.weights==2).sum()

In [ ]:
sample.y.sum(), (~sample.y).sum()

In [ ]:
(sample.weights==0 & sample.y).sum()

In [ ]:
(sample.weights==1 & ~sample.y).sum()

In [ ]:
((sample.weights==2) & sample.y).sum()

In [ ]:
((sample.weights==2) & ~sample.y).sum()

In [ ]:
model = model.to("cuda")
sample = sample.to("cuda")

In [ ]:
with torch.no_grad():
    outputs = model.shared_evaluation(sample, 0)

In [ ]:
outputs

In [ ]:
outputs["all_truth"].sum()

In [ ]:
outputs["target_truth"].sum()

In [ ]:
sample.y.sum()

In [ ]:
preds = torch.sigmoid(out) > 0.5

In [ ]:
# Positives
edge_positive = preds.sum().float()

# Signal true & signal tp
sig_true = outputs["target_truth"].sum().float()
sig_true_positive = (outputs["target_truth"].bool() & preds).sum().float()
background_true_positive = (outputs["all_truth"].bool() & preds).sum().float()

In [ ]:
sig_eff = sig_true_positive / sig_true
sig_pur = sig_true_positive / edge_positive
background_pur = background_true_positive / edge_positive

In [ ]:
sig_eff, sig_pur, background_pur

## Filter Debug

In [ ]:
from gnn4itk_cf.stages.edge_classifier.models.filter import Filter

In [ ]:
config = yaml.load(open('filter_train.yaml'), Loader=yaml.FullLoader)

In [ ]:
model = Filter(config)

In [ ]:
model.setup('fit')

In [ ]:
sample = model.trainset[0]

In [ ]:
sample

In [ ]:
graph_sizes = [sample.edge_index.shape[1] for sample in model.trainset]

In [ ]:
sns.distplot(graph_sizes)

### Metric Debug

In [ ]:
%%time
with torch.no_grad():
    out = model.to("cuda")(sample.to("cuda"))

In [ ]:
sns.distplot(torch.sigmoid(out).cpu())

In [ ]:
sample

In [ ]:
sample.weights.shape

In [ ]:
(sample.weights==-1).sum(), (sample.weights==0).sum(), (sample.weights==1).sum(), (sample.weights==2).sum()

In [ ]:
sample.y.sum(), (~sample.y).sum()

In [ ]:
((sample.weights==1) & sample.y).sum()

In [ ]:
((sample.weights==1) & ~sample.y).sum()

In [ ]:
((sample.weights==0) & sample.y).sum()

In [ ]:
((sample.weights==0) & ~sample.y).sum()

In [ ]:
model = model.to("cuda")
sample = sample.to("cuda")

In [ ]:
with torch.no_grad():
    outputs = model.shared_evaluation(sample, 0)

In [ ]:
outputs

In [ ]:
out = outputs["output"]

In [ ]:
outputs["all_truth"].sum()

In [ ]:
outputs["target_truth"].sum()

In [ ]:
sample.y.sum()

In [ ]:
preds = torch.sigmoid(out) > 0.4

In [ ]:
# Positives
edge_positive = preds.sum().float()

# Signal true & signal tp
sig_true = outputs["target_truth"].sum().float()
sig_true_positive = (outputs["target_truth"].bool() & preds).sum().float()
background_true_positive = (outputs["all_truth"].bool() & preds).sum().float()

In [ ]:
sig_eff = sig_true_positive / sig_true
sig_pur = sig_true_positive / edge_positive
background_pur = background_true_positive / edge_positive

In [ ]:
sig_eff, sig_pur, background_pur

### Normalize inputs

In [ ]:
# Get max, mean and min of each node feature
node_features = model.hparams["node_features"]

In [ ]:
node_features

In [ ]:
statistics = {}
for data in model.trainset:
    for feature in node_features:
        if feature not in statistics:
            statistics[feature] = {"max": [], "mean": [], "min": [], "std": []}
        statistics[feature]["max"].append(data[feature].max())
        statistics[feature]["mean"].append(data[feature].mean())
        statistics[feature]["min"].append(data[feature].min())
        statistics[feature]["std"].append(data[feature].std())

# Get max, mean and min of each statistic feature
for feature in statistics:
    statistics[feature]["max"] = np.max(statistics[feature]["max"])
    statistics[feature]["mean"] = np.mean(statistics[feature]["mean"])
    statistics[feature]["min"] = np.min(statistics[feature]["min"])
    statistics[feature]["std"] = np.mean(statistics[feature]["std"])

In [ ]:
for feature in statistics:
    print(f"{feature} max: {statistics[feature]['max']:.2f}")
    print(f"{feature} mean: {statistics[feature]['mean']:.2f}")
    print(f"{feature} min: {statistics[feature]['min']:.2f}")
    print(f"{feature} std: {statistics[feature]['std']:.2f}")
    print("---")

### Inspect true edges

In [ ]:
sample = model.trainset[0]

In [ ]:
sample

In [ ]:
# Plot true edges in sample for one particle
unique_pids, counts_pids = sample.particle_id.unique(return_counts = True)

In [ ]:
sample_pid = unique_pids[counts_pids >= 10][0]

In [ ]:
sample.particle_id == sample_pid

In [ ]:
sample_true_edges = sample.track_edges[:, sample.particle_id == sample_pid]

In [ ]:
x, y = sample.x_r*1000 * np.cos(sample.x_phi*3.14), sample.x_r*1000 * np.sin(sample.x_phi*3.14)

In [ ]:
sample_true_edges

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x, y, s=0.1, c="black", alpha=0.1)
plt.scatter(x[sample_true_edges[0]], y[sample_true_edges[0]], s=10, c="b")
plt.scatter(x[sample_true_edges[1]], y[sample_true_edges[1]], s=10, c="b")

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x, y, s=0.1, c="black", alpha=0.1)
plt.plot(x[sample_true_edges], y[sample_true_edges], c="b");

In [ ]:
sample

### Debug Reversed Edges

In [ ]:
sample

In [ ]:
%%time
with torch.no_grad():
    out = model.to("cuda")(sample.to("cuda"))

In [ ]:
sns.distplot(torch.sigmoid(out).cpu())

In [ ]:
sample

In [ ]:
sample.weights.shape

In [ ]:
(sample.weights==-1).sum(), (sample.weights==0).sum(), (sample.weights==1).sum(), (sample.weights==2).sum()

In [ ]:
sample.y.sum(), (~sample.y).sum()

In [ ]:
((sample.weights==1) & sample.y).sum()

In [ ]:
((sample.weights==1) & ~sample.y).sum()

In [ ]:
((sample.weights==0) & sample.y).sum()

In [ ]:
((sample.weights==0) & ~sample.y).sum()

In [ ]:
model = model.to("cuda")
sample = sample.to("cuda")

In [ ]:
with torch.no_grad():
    outputs = model.shared_evaluation(sample, 0)

In [ ]:
outputs

In [ ]:
out = outputs["output"]

In [ ]:
outputs["all_truth"].sum()

In [ ]:
outputs["target_truth"].sum()

In [ ]:
sample.y.sum()

In [ ]:
preds = torch.sigmoid(out) > 0.2

In [ ]:
# Positives
edge_positive = preds.sum().float()

# Signal true & signal tp
sig_true = outputs["target_truth"].sum().float()
sig_true_positive = (outputs["target_truth"].bool() & preds).sum().float()
background_true_positive = (outputs["all_truth"].bool() & preds).sum().float()

In [ ]:
sig_eff = sig_true_positive / sig_true
sig_pur = sig_true_positive / edge_positive
background_pur = background_true_positive / edge_positive

In [ ]:
sig_eff, sig_pur, background_pur

Debug the reversing process

In [ ]:
sample

In [ ]:
edge_index = torch.cat([sample.edge_index, sample.edge_index.flip(0)], dim=1)
y = torch.cat([sample.y, sample.y], dim=0)
weights = torch.cat([sample.weights, sample.weights], dim=0)

In [ ]:
edge_index

In [ ]:
y

In [ ]:
weights

In [ ]:
# Edges and y are the same shape, I want to get unique edges and map the uniqueness to the y and weights
unique_edges, unique_edge_indices = torch.unique(edge_index, dim=1, return_inverse=True)
unique_y = torch.zeros_like(unique_edges[0], dtype=y.dtype).scatter(0, unique_edge_indices, y)
unique_weights = torch.zeros_like(unique_edges[0], dtype=weights.dtype).scatter(0, unique_edge_indices, weights)

In [ ]:
unique_weights

In [ ]:
unique_edges

In [ ]:
unique_y

In [ ]:
edge_index[:, y]

In [ ]:
weights[y]

In [ ]:
unique_edges[:, unique_y]

In [ ]:
unique_weights[unique_y]

## Test Figure of Merit Setup

In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=1)

In [ ]:
# Train filter model
trainer.fit(model)

In [ ]:
model.logger